In [1]:
import numpy as np
import tensorflow as tf


In [2]:
!pip install pandas-datareader

In [3]:
import math
import random
import pandas as pd
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

# สร้าง โครงข่าย ai 
## อันดับแรก เราสร้าง class AI_trader () 

In [4]:

class AI_trader ():
    
    #function หลักของ คลาส 
    def __init__(self,state_size,action_num = 3,model_name="AItrader"):#มีสาม การกระทำ หยุด ซื้อ ขาย
        
        self.state      = state_size        #กำหนดสเตดของคลาสนี้
        self.action_num = action_num        #กำหนด จำนวนการกระทำ เท่ากับ 3 คือ หยุด ซื้อ ขาย
        self.memory     = deque(maxlen=2000) #กำหนดให้ deque เมือ max length เท่ากับ 2000 หรือ กำหนดขนาดข้อมูล
        self.inventory  = []                #กำหนด พื้นที่เก็บข้อมูล
        self.model_name = model_name        #กำหนดชื่อโมเดล
        
        self.gamma = 0.95           #กำหนดการลด ค่า value  ของเส้นทาง เท่ากับ 0.05
        self.epsilon = 1.0          #กำหนด ให้ระยะเริ่มแรก ทำ มั่วๆ ไป
        self.epsilon_final = 0.01   #กำหนดให้ระยะหลัง เริ่มทำมั่วๆแค่ 10 %
        self.epsilon_decay = 0.995  #กำหนดให้ลดการกระทำมั่ว ที่ ละ 0.995 * epsilon
        
        self.model = self.สร้างโมเดล() #ทำการ call function สร้างโมเดล() และ ประกาศ ตัวแปร model
        
    #function สำหรับ สร้าง โมเดลของ ai     
    def สร้างโมเดล(self):
        
        model = tf.keras.models.Sequential()#ประกาศโมเดลแบบ ต่อเนื่องขึ้นมา (Sequential)
        #เพิ่มชั้นเลเยอร์ ให้กับโมเดลโดยกำหนดให้รับ data มาจาก input เฉพาะ ชั้นแรก
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state))
        #เพิ่มชั้นเลเยอร์ ที่ 2 ให้กับโมเดล ให้ มีเซลเพิ่มขึ้น จาก 32 ไป 64 
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        #เพิ่มชั้นเลเยอร์ ที่ 3 ให้กับโมเดล ให้ มีเซลเพิ่มขึ้น จาก 64 ไป 128 
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        #เพิ่มชั้นเลเยอร์ตัวสุดท้ายสำหรับกำหนดการกระทำให้กับ bot  ซึ่ง มี สาม การกระทำ หยุด ซื้อ ขาย
        model.add(tf.keras.layers.Dense(units=self.action_num , activation='linear'))
        #ตัวคำนวณให้กับ โครงสร้างของโมเดล โดยให้ loss = mse  optimizers = adam โดยที่ learning rate(ความเร็วในการเรียรู้)= 0.001
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))          
        
        return model  #ส่งโมเดลกลับ
    
    
    def เทรด(self,state):
        #ในfunction นี้ เราจะทำการ รับ state และทำสิ่ง ต่างๆ(มีสาม การกระทำ หยุด ซื้อ ขาย) ตามที่ได้รับค่าstate มา 
        
        #รับค่าสุ่มมาแล้ว ทำการ เทียบ ว่าน้อยกว่าหรือเท่ากับ ค่า  epsilon
        if (random.random() <= self.epsilon) : #function random.random() ทำงานโดยส่งค่า floatกลับ มา  โดยค่านั้นจะอยู่ระหว่าง  [0.0, 1.0)  ::: รายละเอียดการทำงาน https://docs.python.org/3/library/random.html
            return random.randrange(self.action_num)  #ส่งค่า สุ่มมา ในระหว่าง 0 ถึง 2 ซึ่งเป็นสาม ตัวเลข
        
        actions = self.model.predict(state)
        return np.argmax(actions[0])
    
    def batch_train(self,ขนาด_batch): #function สำหรับ learning แบบ custom learning
        batch =[]                     #อันดับแรกกำหนดให้ batch เป็นarray ค่าว่าง 
        for i in range(len(self.memory) - ขนาด_batch + 1,len(self.memory)):  #กำหนด i สำหรับ gradiet decent : อะไร คือ gradiet decent ???:; url:=:>>> https://machinelearningmastery.com/gradient-descent-for-machine-learning/
            
            batch.append(self.memory[i])  #นำเพิ่ม batch ใน memory ตำแหน่ง i  
            
        for state,action,reward,next_state,done in batch:  #กำหนด state,action,reward,next_state,done ให้อยู่ใน batch
            reward = reward                                #update ค่า reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0]) #update ค่า reward แต่อยู่ในเส้นทางอยู่ ยังไม่เสร็จสิ้น การเทรน batch นั้น
                
                
            target = self.model.predict(state)              #กำหนดเป้าหมาย (เส้นชัย)
            
            target[0][action] = reward                      #update target 
            
            self.model.fit(state,target,epochs=1,verbose=0) #เทรน โมเดล 1 เสต็ป สำหรับ batch นั้นๆ   
            
        if self.epsilon > self.epsilon_final:               #กำหนดให้ ลดการสุ่มการกระทำลง
            
            self.epsilon *= self.epsilon_decay
        

# สร้าง ฟังชั่นตัวช่วย 

In [5]:

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

#Price format function
def stocks_price_format(n): #return ค่า โดยเพิ่มว่าเป็นลบหรือบวก
    if n < 0:
        return "- $ {0:2f}".format(abs(n))

    else:
        return "$ {0:2f}".format(abs(n))
    

# ต่อไปเรามาสร้างฟังชั่นสำหรับโหลด ข้อมูลกันเถอะ

In [6]:
def โหลดข้อมูล(ชื่อ_ข้อมูล):
    dataset = data_reader.DataReader(ชื่อ_ข้อมูล,data_source="yahoo") #กำหนดให้ ตัวแปร dataset เป็น ตัวอ่านข้อมูล
    
    start_date = str(dataset.index[0]).split()[0] #นำตัวอ่านข้อมูลมาอ่านข้อมูลวันแรกของด้าต้า แล้วนำมาใส่ตัวแปร start_date
    end_date = str(dataset.index[-1]).split()[0]  #นำตัวอ่านข้อมูลมาอ่านข้อมูลวันสุดท้ายของด้าต้า แล้วนำมาใส่ตัวแปร end_date
    
    close = dataset['Close']
    
    return close
    

In [7]:
def state_creator(data,timestep,window_size):
    
    
    starting_id = timestep - window_size + 1
  
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state])
    

In [8]:
#มาโหลด data กันเถอะ

stock_name = "AAPL"
data = โหลดข้อมูล(stock_name)

In [9]:
data

Date
2015-02-19    128.449997
2015-02-20    129.500000
2015-02-23    133.000000
2015-02-24    132.169998
2015-02-25    128.789993
                 ...    
2020-02-11    319.609985
2020-02-12    327.200012
2020-02-13    324.869995
2020-02-14    324.950012
2020-02-18    319.000000
Name: Close, Length: 1258, dtype: float64

## สร้างโมเดล

In [10]:
#สร้างตัวแปร ที่จำเป็น    ( hyper parameter )
window_size = 10
episodes = 1000

ขนาด_batch = 32
data_samples = len(data) - 1

In [11]:
#ประกาศโมเดล

trader = AI_trader(window_size) #โมเดล ของเรา ชื่อ trader 

In [12]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# สร้าง วงจรการ ฝึกของ a.i.

In [ ]:
for episodes in range(1, episodes + 1) :
    print("Episode : {}/{}".format(episodes,episodes))
    
    state= state_creator(data, 0 , window_size + 1)
    
    
    กำไรรวม = 0
    trader.inventory=[]
    
    for t in tqdm(range(data_samples)):
        
        action= trader.เทรด(state)
        
        next_state = state_creator(data, t+1 , window_size+1)
        
        reward=0
        
        if action == 1 :
            trader.inventory.append(data[t])
            print("AI trader ได้ทำการ ซื้อจำนวน : ",stocks_price_format(data[t]))
            
            
            
        elif action == 2 and len(trader.inventory) > 0 :
            ราคาซื้อ = trader.inventory.pop(0)
            
            reward = max(data[t]- ราคาซื้อ, 0)
            
            กำไรรวม += data[t]- ราคาซื้อ
            print("AI trader ได้ทำการ ขายจำนวน : ",stocks_price_format(data[t]),
                  "กำไร : " + stocks_price_format(data[t] - ราคาซื้อ))
        
        if t == data_samples - 1:
            done = True
        else:
            done = False
        
        trader.memory.append((state,action , reward , next_state ,done))
        state = next_state
        
        if done:
            print("########################")
            print("เงินที่เหลือทั้งหมด: {}".format(เงินที่เหลือ))
            print("########################")
        
        if len(trader.memory) > ขนาด_batch :
            trader.batch_train(ขนาด_batch)
            
    if episodes % 10 == 0 :
        trader.model.save("ai_trader_{}.h5".format(episodes))
        
        
            
            

  0%|                                                                                         | 0/1257 [00:00<?, ?it/s]

Episode : 1/1
AI trader ได้ทำการ ซื้อจำนวน :  $ 129.500000
AI trader ได้ทำการ ซื้อจำนวน :  $ 133.000000
AI trader ได้ทำการ ขายจำนวน :  $ 132.169998 กำไร : $ 2.669998
AI trader ได้ทำการ ซื้อจำนวน :  $ 128.789993
AI trader ได้ทำการ ขายจำนวน :  $ 130.419998 กำไร : - $ 2.580002
AI trader ได้ทำการ ซื้อจำนวน :  $ 128.460007
AI trader ได้ทำการ ขายจำนวน :  $ 128.539993 กำไร : - $ 0.250000
AI trader ได้ทำการ ซื้อจำนวน :  $ 126.410004
AI trader ได้ทำการ ซื้อจำนวน :  $ 126.599998
AI trader ได้ทำการ ซื้อจำนวน :  $ 127.139999
AI trader ได้ทำการ ซื้อจำนวน :  $ 122.239998
AI trader ได้ทำการ ขายจำนวน :  $ 124.449997 กำไร : - $ 4.010010
AI trader ได้ทำการ ขายจำนวน :  $ 123.589996 กำไร : - $ 2.820007
AI trader ได้ทำการ ซื้อจำนวน :  $ 128.470001
AI trader ได้ทำการ ซื้อจำนวน :  $ 127.500000
AI trader ได้ทำการ ซื้อจำนวน :  $ 125.900002
AI trader ได้ทำการ ขายจำนวน :  $ 123.250000 กำไร : - $ 3.349998
AI trader ได้ทำการ ซื้อจำนวน :  $ 126.370003
AI trader ได้ทำการ ซื้อจำนวน :  $ 124.250000


  3%|██▏                                                                             | 34/1257 [00:08<23:31,  1.15s/it]

AI trader ได้ทำการ ซื้อจำนวน :  $ 126.559998


  3%|██▎                                                                             | 37/1257 [00:17<48:51,  2.40s/it]

...

  9%|███████▎                                                                     | 119/1257 [11:53<7:17:08, 23.05s/it]

AI trader ได้ทำการ ซื้อจำนวน :  $ 119.720001


 10%|███████▎                                                                     | 120/1257 [12:01<5:51:39, 18.56s/it]

AI trader ได้ทำการ ขายจำนวน :  $ 113.489998 กำไร : - $ 11.010002


 10%|███████▍                                                                     | 121/1257 [12:07<4:39:37, 14.77s/it]